In [47]:
import re
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm


In [48]:
def news_function(category, start, end, page, header):
    """
    input은 4개입니다.
    카테고리는 '검색 이름'
    start는 시작날짜 : '20221101' -> 22년 11월 01일부터 검색
    end는 종료날짜 : '20221203' -> 22년 12월 03일까지 검색
    page는 몇개의 페이지를 저장할건지 : 3 -> 3페이지
    header는 크롬에서 F12 누르고 Network -> All -> Headers -> User-Agent 복사해서 dict형태로 넣어주면 됩니다.

    ex) 
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    news_function('음식료품','20221101','20221203',3, header) 
    -> 음식료품을 검색해서 22년 11월 1일부터 22년 12월 3일까지 3페이지씩 저장한다.

    개인에 맞게 변경해야되는 것은 to_csv 주소입니다. 이것은 개인 컴퓨터에 맞게 변경해야됩니다.
    """

    query = category
    header = header 
    
    date_range = pd.date_range(start, end)
    
    news_title = []
    for date in tqdm(date_range):
        date_str = date.strftime('%Y%m%d')
        for page_num in range(page):
            start_idx = page_num * 10 + 1
            url = f'https://search.naver.com/search.naver?where=news&query={query}&sort=0&nso=so%3Ar%2Cp%3Afrom{date_str}to{date_str}&start={start_idx}'
            res = requests.get(url, headers=header)
            soup = bs(res.text, 'lxml')
            for title in soup.find_all('a', class_='news_tit'):
                news_title.append([date_str, title['title']])
                
    news_title = pd.DataFrame(news_title).drop_duplicates()
    news_title['category'] = query
    news_title.columns = ['time', 'title', 'category']
    #news_title.to_csv('../Web_Crawling/save_data/' + query + '_total.csv', encoding='utf-8-sig', index=False) # csv 내보내기
    return news_title

In [49]:
news_function('빗길사고','20220101', '20220110', 1, {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'} )

100%|██████████| 10/10 [00:02<00:00,  3.86it/s]


,time,title,category
0,20220102,[르포] 이준석 닷새 머문 상주안전체험교육센터...300m 빙판길 올랐더니,빗길사고
1,20220103,사망사고 내고…운전 안했다던 고교생 결국,빗길사고
2,20220104,[손바닥문학상 가작 수상작] 화이불변,빗길사고
3,20220107,“중년의 허전함 달래준 건… 10년만에 다시 잡은 클럽”,빗길사고


In [50]:
def get_articles(query, start, end, page):
    link_save = []
    
    date_range = pd.date_range(start, end)
    
    for date in tqdm(date_range):
        date_str = date.strftime('%Y%m%d')
        for page_num in range(page):
            start_idx = page_num * 10 + 1
            search_url = f'https://search.naver.com/search.naver?where=news&query={query}&sort=0&nso=so%3Ar%2Cp%3Afrom{date_str}to{date_str}&start={start_idx}'
        
        
            response = requests.get(search_url)
            soup = bs(response.text, "html.parser")
            articles = soup.find_all("div", class_="news_area")

            for article in articles:
                title = article.find("a", class_="news_tit").text.strip()
                link = article.find("a", class_="news_tit")["href"]
                link_save.append(link)
                article_response = requests.get(link)
                article_soup = bs(article_response.text, "html.parser")

                content = article_soup.find("div", class_="news_end")
                if content is not None:
                    content = content.text.strip()
                else:
                    content = "본문을 가져올 수 없습니다."

                date_articles = article_soup.find("span", class_="t11")
                date_articles_element = article_soup.find("span", class_="t11")
                if date_articles_element is not None:
                    date_articles = date_articles_element.text.strip()
                else:
                    date_articles = "날짜를 가져올 수 없습니다."

                # print("제목:", title)
                # print("본문:", content)
                # print("날짜:", date_articles)
                # print()

    return link_save, title, content, date_articles

In [51]:
category = '빗길사고'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
start = '20210101'
end = '20210105'
page = 1

In [52]:
get_articles(category, start, end, page)

100%|██████████| 5/5 [00:02<00:00,  1.80it/s]


(['https://www.iloveorganic.co.kr/news/articleView.html?idxno=318339',
  'http://star.mk.co.kr/new/view.php?mc=ST&year=2021&no=4800',
  'http://www.queen.co.kr/news/articleView.html?idxno=349536',
  'http://www.worklaw.co.kr/view/view.asp?accessSite=Naver&accessMethod=Search&accessMenu=News&in_cate=119&in_cate2=0&gopage=1&bi_pidx=31758'],
 '배달음식이 조금 늦게 오더라도',
 '본문을 가져올 수 없습니다.',
 '날짜를 가져올 수 없습니다.')